In [1]:
%matplotlib inline
%load_ext heat

In [ ]:
#%%heat

from IPython.display import Image
from copy import copy,deepcopy
from numpy import pi
import numpy as np
from qutip import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit, Gate
import time
from itertools import combinations

def user_gate1():
# S gate
    mat = np.array([[1.,0],[0., 1.j]])
    return Qobj(mat, dims=[[2], [2]])

def user_gate2():
# T gate
    mat = np.array([[1.,0],[0., 0.707+0.707j]])
    return Qobj(mat, dims=[[2], [2]])

def qu(gnum,n): # pick the unitary from the gate set #error
    
    if gnum<2*n+1:
        if gnum%2==0:
            g_t= Gate("T",int((gnum-1)//2)) 
        else:
            g_t= Gate("SNOT", int((gnum-1)//2))
    elif gnum<2*n+n+1:
        #print((gnum-3*n)%n, (gnum-3*n+1)%n )
        g_t= Gate("CNOT",int((gnum-2*n)%n),int((gnum-2*n+1)%n))
    return g_t

def quc(x,n): # generate the given quantum circuit
    qc= QubitCircuit(n)
    for i in range(len(x)):
        if x[i]!=0:
            qc.add_gate(qu(x[i], n))
    prop = qc.propagators()
    return gate_sequence_product(prop).full()
    

#may be faster to apply inverse i.e. loop over numbers gmax**r in base gmax
# def list_circuits(gmax,r):# list the circuits
#     def list_c(*arrays): 
#         grid = np.meshgrid(*arrays)        
#         coord_list = [entry.ravel() for entry in grid]
#         points = np.vstack(coord_list).T
#         return points
#     aa = np.arange(gmax)
#     return list_c(*r*[np.arange(gmax)])

# below is time inefficient but memory efficient
# not optimal 28.8 vs 15.7

def base_conv(cirnum,gmax,r): 
    xx= np.zeros(r)
    num=np.copy(cirnum)
    for i in range(r): 
        xx[i]= num%gmax
        num//=gmax
    return xx



def bi(ii,n):
    bix = np.zeros(n)
    for i in range(n): 
        bix[i]= ii%2
        ii//=2
    return(bix)

def list_povm(n): 
    l_povms=[]
    for i in range(2**n):
        pv = np.zeros(2**n)
        b = bi(i,n)
        pv[0]= 1
        for j in range(len(b)): 
            if b[j]==1:
                pv[2**j:2**(j+1)]= pv[:2**j]
        l_povms.append(np.diag(pv))
    return(l_povms)


# below is time inefficient but memory efficient
#removed since, it is less optimal 4.67s vs 3.54s


def gene_povm(gnp,n): 
    pv = np.zeros(2**n)
    b = gnp
    pv[0]= 1
    for j in range(len(b)): 
        if b[j]==1:
            pv[2**j:2**(j+1)]= pv[:2**j]
    return(np.diag(pv)) 

            
                
                

def trqrho(q,rho):
    trq= 0
    trq+= np.sum(np.diag(q)*np.diag(rho),)
    return(trq)

def tr(q): 
    return(np.sum(q))


def H_ee(r,eta,rho,gmax,n):
    ndg_list=[]
    tt=0
        
    for cnum in range(0,gmax**r):# this checks for uniqueness of the given gate
        bc=base_conv(cnum,gmax,r)
        if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):
            #print(cnum)
            #print(bc)
            ndg_list = np.append(ndg_list,cnum)
            #print(ndg_list)
    circi=0
    povmi=np.zeros(n)
    H=2**n
    for cnum in np.nditer(ndg_list):
        if cnum==0:
            counter = 0
            for qbits in range(n+1): 
                for pv in combinations(np.arange(n),n-qbits):
                    pov = np.zeros(n)
                    for iden in  range(len(pv)):
                        pov[pv[iden]]= 1
                    povm = gene_povm(pov,n)
                    trqq=trqrho(rho,povm)
                    if trqq>eta or trqq==eta: 
                        counter+=1
                        if tr(povm)<H:
                            circi = cnum
                            povmi = pov
                            H = tr(povm)
                if counter==0:
                    break
                else: 
                    counter=0

            
        else:
            #print(base_conv(cnum,gmax,r))
            bc = base_conv(cnum,gmax,r)
            U = quc(bc,n)
            #print(U)
            sigma = U@rho@U.conj() # use of einsum may make it faster? need to make this step faster? sparce?
            counter = 0
            for qbits in range(n+1): 
                for pv in combinations(np.arange(n),n-qbits):
                    pov = np.zeros(n)
                    for iden in  range(len(pv)):
                        pov[pv[iden]]= 1
                    povm = gene_povm(pov,n)
                    trqq=trqrho(sigma,povm)
                    if trqq>eta or trqq==eta: 
                        counter+=1
                        if tr(povm)<H:
                            circi = cnum
                            povmi = pov
                            H = tr(povm)
                if counter==0:
                    break
                else: 
                    counter=0
    
    print("effective entropy for eta="+str(eta)+" and r="+str(r) +" is "+ str(np.log(H)/np.log(2)))
    print(povmi)
    print(circi)
    return(np.log(H)/np.log(2))
# All the inputs are in this box 

# system specific data
n = 3 # number of qubits
gmax= 1+2*n+n #number of unique gates in the universal gate set [2*n 1-qubit gates, n CNOT gates]
# Need to change qu() if this is changed

# process specific

In [ ]:
#%%heat

from IPython.display import Image
from copy import copy,deepcopy
from numpy import pi
import numpy as np
from qutip import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit, Gate
import time
from itertools import combinations

def user_gate1():
# S gate
    mat = np.array([[1.,0],[0., 1.j]])
    return Qobj(mat, dims=[[2], [2]])

def user_gate2():
# T gate
    mat = np.array([[1.,0],[0., 0.707+0.707j]])
    return Qobj(mat, dims=[[2], [2]])

def qu(gnum,n): # pick the unitary from the gate set #error
    
    if gnum<2*n+1:
        if gnum%2==0:
            g_t= Gate("T",int((gnum-1)//2)) 
        else:
            g_t= Gate("SNOT", int((gnum-1)//2))
    elif gnum<2*n+n+1:
        #print((gnum-3*n)%n, (gnum-3*n+1)%n )
        g_t= Gate("CNOT",int((gnum-2*n)%n),int((gnum-2*n+1)%n))
    return g_t

def quc(x,n): # generate the given quantum circuit
    qc= QubitCircuit(n)
    for i in range(len(x)):
        if x[i]!=0:
            qc.add_gate(qu(x[i], n))
    prop = qc.propagators()
    return gate_sequence_product(prop).full()
    

#may be faster to apply inverse i.e. loop over numbers gmax**r in base gmax
# def list_circuits(gmax,r):# list the circuits
#     def list_c(*arrays): 
#         grid = np.meshgrid(*arrays)        
#         coord_list = [entry.ravel() for entry in grid]
#         points = np.vstack(coord_list).T
#         return points
#     aa = np.arange(gmax)
#     return list_c(*r*[np.arange(gmax)])

# below is time inefficient but memory efficient
# not optimal 28.8 vs 15.7

def base_conv(cirnum,gmax,r): 
    xx= np.zeros(r)
    num=np.copy(cirnum)
    for i in range(r): 
        xx[i]= num%gmax
        num//=gmax
    return xx



def bi(ii,n):
    bix = np.zeros(n)
    for i in range(n): 
        bix[i]= ii%2
        ii//=2
    return(bix)

def list_povm(n): 
    l_povms=[]
    for i in range(2**n):
        pv = np.zeros(2**n)
        b = bi(i,n)
        pv[0]= 1
        for j in range(len(b)): 
            if b[j]==1:
                pv[2**j:2**(j+1)]= pv[:2**j]
        l_povms.append(np.diag(pv))
    return(l_povms)


# below is time inefficient but memory efficient
#removed since, it is less optimal 4.67s vs 3.54s


def gene_povm(gnp,n): 
    pv = np.zeros(2**n)
    b = gnp
    pv[0]= 1
    for j in range(len(b)): 
        if b[j]==1:
            pv[2**j:2**(j+1)]= pv[:2**j]
    return(np.diag(pv)) 

            
                
                

def trqrho(q,rho):
    trq= 0
    trq+= np.sum(np.diag(q)*np.diag(rho),)
    return(trq)

def tr(q): 
    return(np.sum(q))


def H_ee(r,eta,rho,gmax,n):
    ndg_list=[]
    tt=0
        
    for cnum in range(0,gmax**r):# this checks for uniqueness of the given gate
        bc=base_conv(cnum,gmax,r)
        if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):
            #print(cnum)
            #print(bc)
            ndg_list = np.append(ndg_list,cnum)
            #print(ndg_list)
    circi=0
    povmi=np.zeros(n)
    H=2**n
    for cnum in np.nditer(ndg_list):
        if cnum==0:
            counter = 0
            for qbits in range(n+1): 
                for pv in combinations(np.arange(n),n-qbits):
                    pov = np.zeros(n)
                    for iden in  range(len(pv)):
                        pov[pv[iden]]= 1
                    povm = gene_povm(pov,n)
                    trqq=trqrho(rho,povm)
                    if trqq>eta or trqq==eta: 
                        counter+=1
                        if tr(povm)<H:
                            circi = cnum
                            povmi = pov
                            H = tr(povm)
                if counter==0:
                    break
                else: 
                    counter=0

            
        else:
            #print(base_conv(cnum,gmax,r))
            bc = base_conv(cnum,gmax,r)
            U = quc(bc,n)
            #print(U)
            sigma = U@rho@U.conj() # use of einsum may make it faster? need to make this step faster? sparce?
            counter = 0
            for qbits in range(n+1): 
                for pv in combinations(np.arange(n),n-qbits):
                    pov = np.zeros(n)
                    for iden in  range(len(pv)):
                        pov[pv[iden]]= 1
                    povm = gene_povm(pov,n)
                    trqq=trqrho(sigma,povm)
                    if trqq>eta or trqq==eta: 
                        counter+=1
                        if tr(povm)<H:
                            circi = cnum
                            povmi = pov
                            H = tr(povm)
                if counter==0:
                    break
                else: 
                    counter=0
    
    print("effective entropy for eta="+str(eta)+" and r="+str(r) +" is "+ str(np.log(H)/np.log(2)))
    print(povmi)
    print(circi)
    return(np.log(H)/np.log(2))

In [57]:
# All the inputs are in this box 

# system specific data
n = 3 # number of qubits
gmax= 1+2*n+n #number of unique gates in the universal gate set [2*n 1-qubit gates, n CNOT gates]
# Need to change qu() if this is changed

# process specific

eta = 0.8 # success rate? 
# define rho below this
x = np.random.rand(2**n,2**n)
rho= x@x.transpose()
rho = rho/np.trace(rho)
#H_ee(r,eta,rho,gmax,n)
np.savetxt("rho1.txt",rho)

In [ ]:
eta = 0.80 
rr = np.zeros((8,2))
for i in range(len(rr)):
    r= i
    rho = np.loadtxt("rho1.txt")
    print(np.shape(rho))
    rr[i,0]=i
    rr[i,1]= H_ee(r,eta,rho,gmax,n)
    np.savetxt('rr1.txt',rr)
print(rr)
np.savetxt('rr1.txt',rr)
    
    
    

(8, 8)
effective entropy for eta=0.8 and r=0 is 3.0
[0. 0. 0.]
0
(8, 8)
effective entropy for eta=0.8 and r=1 is 2.0
[1. 1. 0.]
1.0
(8, 8)
effective entropy for eta=0.8 and r=2 is 1.0
[1. 0. 0.]
13.0
(8, 8)
effective entropy for eta=0.8 and r=3 is 1.0
[1. 0. 0.]
13.0
(8, 8)
effective entropy for eta=0.8 and r=4 is 1.0
[1. 0. 0.]
13.0
(8, 8)
effective entropy for eta=0.8 and r=5 is 1.0
[1. 0. 0.]
13.0
(8, 8)
effective entropy for eta=0.8 and r=6 is 1.0
[1. 0. 0.]
13.0
(8, 8)


In [50]:
etan = np.zeros((7,2))
x = np.linspace(0.6,0.95,len(etan))
r = 5
for i in range(len(etan)):
    eta = np.copy(x[i])
    etan[i,0]=np.copy(x[i])
    etan[i,1]= H_ee(r,eta,rho,gmax,n)
    np.savetxt("eta1.txt",etan)
np.savetxt("eta1.txt",etan)
    

effective entropy for eta=0.6 and r=5 is 0.0
[0. 0. 0.]
135.0
effective entropy for eta=0.6583333333333333 and r=5 is 0.0
[0. 0. 0.]
135.0
effective entropy for eta=0.7166666666666667 and r=5 is 0.0
[0. 0. 0.]
135.0
effective entropy for eta=0.7749999999999999 and r=5 is 1.0
[1. 0. 0.]
13.0
effective entropy for eta=0.8333333333333333 and r=5 is 2.0
[1. 1. 0.]
1.0
effective entropy for eta=0.8916666666666666 and r=5 is 2.0
[1. 1. 0.]
1.0
effective entropy for eta=0.95 and r=5 is 3.0
[0. 0. 0.]
0


IndexError: index 7 is out of bounds for axis 0 with size 7

In [51]:
len(etan)


7

In [31]:
print(10*((a-1)**(b+1)-1)/(a-2)/a**b)

0.00029881573748535915


In [10]:
print(4*np.log(16)/np.log(10))

4.816479930623698


In [ ]:
print(povmi)#873
print(circi)
U = quc(base_conv(circi,gmax,r),n)
print(np.diag(U@rho@U.conj()))
print(np.sum(np.diag(U@rho@U.conj())))